In [2]:
import numpy as np
import pandas as pd

PART 1

 1.1 MultiIndex Series 

In [3]:
 data = pd.Series([0.93, 0.32, 0.18, 0.20, 0.57, 0.60, 0.96, 0.65, 0.75],
    index=[
        ["a","a","a","b","b","c","c","d","d"],
        [1,2,3,1,3,1,2,2,3]
    ]
 ) 

Extract all values where first-level index = 'b'.

In [4]:
print(data.loc['b'])

1    0.20
3    0.57
dtype: float64


Slice from 'b' to 'c'.

In [5]:
data['b':'c']

b  1    0.20
   3    0.57
c  1    0.60
   2    0.96
dtype: float64

Select all values where second-level index = 2.

In [6]:
print(data.loc[:, 2])

a    0.32
c    0.96
d    0.65
dtype: float64


Unstack the Series.

In [7]:
data.unstack()

,1,2,3
a,0.93,0.32,0.18
b,0.20,NaN,0.57
c,0.60,0.96,NaN
d,NaN,0.65,0.75


Stack back and verify equality.

In [8]:
data.unstack().stack()

a  1    0.93
   2    0.32
   3    0.18
b  1    0.20
   3    0.57
c  1    0.60
   2    0.96
d  2    0.65
   3    0.75
dtype: float64

 1.2 MultiIndex DataFrame

In [9]:
 frame = pd.DataFrame(
    np.arange(12).reshape((4, 3)),
        index=[["a","a","b","b"], [1,2,1,2]],
        columns=[["Ohio","Ohio","Colorado"],
        ["Green","Red","Green"]]
 )

Assign level names.

In [10]:
 frame.index.names = ["key1", "key2"]
 frame.columns.names = ["state", "color"]
 frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

 Select “Ohio” columns.

In [11]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

swaplevel then sort.

In [12]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

Group by index level and column level.

In [13]:
 frame.groupby(level = 'key1').sum() #or key2


state  Ohio     Colorado
color Green Red    Green
key1                    
a         3   5        7
b        15  17       19

In [14]:
frame.groupby(level = 'color', axis = 'columns').sum() #groupby columns

/tmp/ipython-input-1159622431.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  frame.groupby(level = 'color', axis = 'columns').sum() #groupby columns


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

PART 2 — COMBINING AND MERGING DATASETS

2.1 Basic Merge

In [15]:
df1 = pd.DataFrame({
    "key":  ["b", "b", "a", "c", "a", "a", "b"],
    "data1": pd.Series([0, 1, 2, 3, 4, 5, 6], dtype="Int64")
    }
)
 
df2 = pd.DataFrame({
    "key":  ["a", "b", "d"],
    "data2": pd.Series([0, 1, 2], dtype="Int64")
    }
)

Inner, Left, Outer merge.

In [16]:
inner = pd.merge(df1, df2) #inner
print(inner)

  key  data1  data2
0   b      0      1
1   b      1      1
2   a      2      0
3   a      4      0
4   a      5      0
5   b      6      1


In [17]:
left = pd.merge(df1, df2, on = 'key', how = 'left') #how = 'right'
left


,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,c,3,<NA>
4,a,4,0
5,a,5,0
6,b,6,1


In [18]:
outer = pd.merge(df1, df2, how = 'outer')
outer

,key,data1,data2
0,a,2,0
1,a,4,0
2,a,5,0
3,b,0,1
4,b,1,1
5,b,6,1
6,c,3,<NA>
7,d,<NA>,2


Explain row count differences.

In [19]:
print(f"df1: {len(df1)} rows")
print(f"df2: {len(df2)} rows")
print(f"\nInner merge: {len(inner)} rows")
print(f"Left merge: {len(left)} rows")
print(f"Outer merge: {len(outer)} rows")

df1: 7 rows
df2: 3 rows

Inner merge: 6 rows
Left merge: 7 rows
Outer merge: 8 rows


 2.2 Many-to-Many Merge

In [20]:
df1 = pd.DataFrame({
    "key":   ["b", "b", "a", "c", "a", "b"],
    "data1": pd.Series([0, 1, 2, 3, 4, 5], dtype="Int64")
    }
)
 
df2 = pd.DataFrame({
    "key":   ["a", "b", "a", "b", "d"],
    "data2": pd.Series([0, 1, 2, 3, 4], dtype="Int64") 
    }
)

 Perform left merge.


In [21]:
left= pd.merge(df1, df2, on = 'key', how = 'left')
left

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


 • Count rows where key='b'.


In [22]:
print(f"Lefr merge: {(left['key'] == 'b').sum()} rows") 

Lefr merge: 6 rows


 • Explain many-to-many behavior.
 - Trong df1, key = "a" xuất hiện 2 lần, và trong df2 cũng có 2 lần. → Khi merge, mỗi bản ghi của "a" trong df1 sẽ kết hợp với tất cả bản ghi "a" trong df2.
--> kết quả 2*2 = 4 dòng cho "a"
- Với key = "b", df1 có 3 lần, df2 có 2 lần. → Khi merge, ta được 
3*2 = 6 dòng cho "b".
- Với key = "c", chỉ có trong df1, không có trong df2. → Kết quả: dữ liệu data2 bị NaN.
- Với key = "c", chỉ có trong df1, không có trong df2. → Kết quả: dữ liệu data2 bị NaN.


2.3 Merge on Multiple Keys

In [23]:
left = pd.DataFrame({
    "k1":   ["foo", "foo", "bar"],
    "k2":   ["one", "two", "one"],
    "lval": pd.Series([1, 2, 3], dtype="Int64")
    }
)
 
right = pd.DataFrame({
    "k1":   ["foo", "foo", "bar", "bar"],
    "k2":   ["one", "one", "one", "two"],
    "rval": pd.Series([4, 5, 6, 7], dtype="Int64")
    }
)

 • Outer merge.

In [24]:
outer = pd.merge(left, right, how = 'outer')
outer

,k1,k2,lval,rval
0,bar,one,3,6
1,bar,two,<NA>,7
2,foo,one,1,4
3,foo,one,1,5
4,foo,two,2,<NA>


 • Explain NA values.
 - foo, one: có trong cả left và right.

+ left có 1 dòng, right có 2 dòng → tạo ra 2 dòng kết hợp.

+ Không có NA vì cả hai bảng đều có dữ liệu.

- foo, two: chỉ có trong left, không có trong right.

→ rval bị NaN.

- bar, one: có trong cả hai bảng, nên merge thành một dòng đầy đủ.

+ Không có NA.

- bar, two: chỉ có trong right, không có trong left.

→ lval bị NaN.

 • Merge on "k1" with suffixes.


In [25]:
merged = pd.merge(left, right, on="k1", suffixes=("_L", "_R"))
print(merged)

    k1 k2_L  lval k2_R  rval
0  foo  one     1  one     4
1  foo  one     1  one     5
2  foo  two     2  one     4
3  foo  two     2  one     5
4  bar  one     3  one     6
5  bar  one     3  two     7


2.4 Index-Based Joins

In [26]:
left1 = pd.DataFrame({
    "key":   ["a", "b", "a", "a", "b", "c"],
    "value": pd.Series([0, 1, 2, 3, 4, 5], dtype="Int64")
    }
)
 
right1 = pd.DataFrame({
    "group_val": [3.5, 7.0]
    }, index=["a", "b"]
)

 • Merge on index.

In [27]:
merge = pd.merge(left1, right1, left_on="key", right_index=True, how="left")
merge

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


 • Perform join().

In [28]:
joined = left1.set_index("key").join(right1)
#left1.join(right1, on = 'key')
print(joined)

     value  group_val
key                  
a        0        3.5
b        1        7.0
a        2        3.5
a        3        3.5
b        4        7.0
c        5        NaN


 • Compare readability.

- merge: linh hoạt hơn, có thể chỉ định left_on, right_on, left_index, right_index, và nhiều loại join (inner, outer, left, right).

- join: ngắn gọn hơn khi muốn ghép theo index, đặc biệt khi đã set index sẵn.

PART 3 — CONCATENATION

In [29]:
s1 = pd.Series([0, 1], index=["a", "b"], dtype="Int64")
s2 = pd.Series([2, 3, 4], index=["c", "d", "e"], dtype="Int64")
s3 = pd.Series([5, 6], index=["f", "g"], dtype="Int64")

 • Concat row-wise.

In [30]:
pd.concat([s1, s2, s3], axis = 0)

,0
a,0
b,1
c,2
d,3
e,4
f,5
g,6


 • Concat column-wise.

In [31]:
 pd.concat([s1, s2, s3], axis="columns")

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


 • Concat with join='inner'.

In [32]:
pd.concat([s1, s2, s3], axis = 1,  join = 'inner')

,0,1,2


 • Create hierarchical keys and unstack.

In [33]:
concat = pd.concat([s1, s2, s3], keys=["s1", "s2", "s3"])
print(concat)

s1  a    0
    b    1
s2  c    2
    d    3
    e    4
s3  f    5
    g    6
dtype: Int64


In [34]:
print(concat.unstack())

       a     b     c     d     e     f     g
s1     0     1  <NA>  <NA>  <NA>  <NA>  <NA>
s2  <NA>  <NA>     2     3     4  <NA>  <NA>
s3  <NA>  <NA>  <NA>  <NA>  <NA>     5     6


PART 4 — RESHAPING AND PIVOTING

In [35]:
data = pd.DataFrame(
    np.arange(6).reshape((2, 3)),
    index=pd.Index(["Ohio", "Colorado"], name="state"),
    columns=pd.Index(["one", "two", "three"], name="number")
)

 • stack(), unstack().

In [36]:
data.stack().unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [37]:
data.stack()

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [38]:
data.unstack()

number  state   
one     Ohio        0
        Colorado    3
two     Ohio        1
        Colorado    4
three   Ohio        2
        Colorado    5
dtype: int64

 • Unstack on different level.

In [39]:
data.stack().unstack(level = 0) #level = 'state'

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [40]:
data.stack().unstack(level = 1) #level = 'number'

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


 • Explain difference.
 - stack(): “nén” dữ liệu, chuyển từ dạng bảng (wide format) sang dạng Series với MultiIndex (long format).

- unstack(): “mở rộng” dữ liệu, chuyển từ Series MultiIndex sang DataFrame (wide format).

4.2 Pivot

In [41]:
df = pd.DataFrame({
    "date":   ["2021-01", "2021-01", "2021-02", "2021-02"],
    "item":   ["A",       "B",       "A",       "A"],
    "sales":  [10,        20,        15,        18]
    }   
)

• Try pivot().

In [42]:
df.pivot(index="date", columns="item", values="sales")
#xảy ra lỗi
#Với date="2021-02" và item="A", có 2 giá trị sales (15 và 18).
#pivot() yêu cầu mỗi cặp (index, column) phải duy nhất, nhưng ở đây bị trùng.

ValueError: Index contains duplicate entries, cannot reshape

In [43]:
#pivot_table()
df.pivot_table(index="date", columns="item", values="sales", aggfunc="sum")

item,A,B
date,,
2021-01,10.0,20.0
2021-02,33.0,NaN


4.3 Melt

In [44]:
df_wide = pd.DataFrame({
    "customer_id": [1, 2, 3],
    "Jan_sales":   [100, 150, 200],
    "Feb_sales":   [110, 160, 210],
    "Mar_sales":   [120, 170, 220]
    }
)

 • Melt to tidy format.

In [45]:
df_melt = pd.melt(
    df_wide,
    id_vars=["customer_id"],       # giữ nguyên cột định danh
    var_name="month",              # tên cột mới cho biến
    value_name="sales"             # tên cột mới cho giá trị
)
df_melt

,customer_id,month,sales
0,1,Jan_sales,100
1,2,Jan_sales,150
2,3,Jan_sales,200
3,1,Feb_sales,110
4,2,Feb_sales,160
5,3,Feb_sales,210
6,1,Mar_sales,120
7,2,Mar_sales,170
8,3,Mar_sales,220


 • Compare melt vs stack.
- melt(): thường dùng khi muốn chuyển từ wide → long format, đặc biệt cho dữ liệu nhiều cột đo lường.

- stack(): cũng chuyển từ wide → long, nhưng tạo MultiIndex thay vì cột mới.

    + stack() phù hợp khi index có nhiều cấp.

    + melt() phù hợp khi muốn dữ liệu tidy để dùng cho phân tích/ML.

• Explain ML use-case.
- ML thường yêu cầu dữ liệu tidy format: mỗi hàng là một quan sát, mỗi cột là một biến.

- Với df_melt, ta dễ dàng lọc theo tháng, vẽ biểu đồ theo thời gian, hoặc đưa vào mô hình dự đoán doanh số.

PART 5 — MINI PROJECT 

In [46]:
#data provided
transactions = pd.DataFrame({
    "transaction_id": [1, 2, 3, 4, 5, 6],
    "date":           ["2021-01", "2021-01", "2021-02",
                       "2021-02", "2021-03", "2021-03"],
    "customer_id":    [101, 102, 101, 103, 102, 101],
    "product":        ["A", "A", "B", "A", "B", "A"],
    "quantity":       [1, 2, 1, 3, 2, 1],
    "price":          [10.0, 10.0, 20.0, 10.0, 20.0, 10.0]
    }
)

product_dim = pd.DataFrame({
    "product":  ["A", "B"],
    "category": ["Standard", "Premium"]
    }
)

 • Compute revenue.

In [47]:
transactions['revenue'] = transactions['quantity']* transactions['price']
transactions

,transaction_id,date,customer_id,product,quantity,price,revenue
0,1,2021-01,101,A,1,10.0,10.0
1,2,2021-01,102,A,2,10.0,20.0
2,3,2021-02,101,B,1,20.0,20.0
3,4,2021-02,103,A,3,10.0,30.0
4,5,2021-03,102,B,2,20.0,40.0
5,6,2021-03,101,A,1,10.0,10.0


 • Monthly revenue pivot.

In [48]:
monthly_revenue = pd.pivot_table(transactions, index = 'date', values = 'revenue', aggfunc = 'sum')
monthly_revenue

,revenue
date,
2021-01,30.0
2021-02,50.0
2021-03,50.0


 • Merge category.

In [49]:
merged = pd.merge(transactions, product_dim, on = 'product')
merged

,transaction_id,date,customer_id,product,quantity,price,revenue,category
0,1,2021-01,101,A,1,10.0,10.0,Standard
1,2,2021-01,102,A,2,10.0,20.0,Standard
2,3,2021-02,101,B,1,20.0,20.0,Premium
3,4,2021-02,103,A,3,10.0,30.0,Standard
4,5,2021-03,102,B,2,20.0,40.0,Premium
5,6,2021-03,101,A,1,10.0,10.0,Standard


 • Pivot by category.

In [50]:
by_category = merged.pivot_table(
    index="date", columns="category", values="revenue", aggfunc="sum"
)
print(by_category)

category  Premium  Standard
date                       
2021-01       NaN      30.0
2021-02      20.0      30.0
2021-03      40.0      10.0


 • Concatenate new month.

In [52]:
#thêm tháng 4/2021
new_month = pd.DataFrame({
    "transaction_id": [7, 8],
    "date":           ["2021-04", "2021-04"],
    "customer_id":    [104, 105],
    "product":        ["A", "B"],
    "quantity":       [2, 3],
    "price":          [30, 50.0]
    }
)
new_month["revenue"] = new_month["quantity"] * new_month["price"]

transactions_new = pd.concat([transactions, new_month], ignore_index=True)
print(pd.merge(transactions_new, product_dim, on = 'product'))

   transaction_id     date  customer_id product  quantity  price  revenue  \
0               1  2021-01          101       A         1   10.0     10.0   
1               2  2021-01          102       A         2   10.0     20.0   
2               3  2021-02          101       B         1   20.0     20.0   
3               4  2021-02          103       A         3   10.0     30.0   
4               5  2021-03          102       B         2   20.0     40.0   
5               6  2021-03          101       A         1   10.0     10.0   
6               7  2021-04          104       A         2   30.0     60.0   
7               8  2021-04          105       B         3   50.0    150.0   

   category  
0  Standard  
1  Standard  
2   Premium  
3  Standard  
4   Premium  
5  Standard  
6  Standard  
7   Premium  


In [54]:
#montly revenue
monthly_rev_new = transactions_new.pivot_table(
    index="date", values="revenue", aggfunc="sum"
)
print(monthly_rev_new)

         revenue
date            
2021-01     30.0
2021-02     50.0
2021-03     50.0
2021-04    210.0


 • Short interpretation.
- Doanh thu có xu hướng tăng trưởng mạnh, nhờ loại 'Premium'.
- Premium đang trở thành sản phẩm chủ lực, nhất là từ tháng 3–4.
- Standard tăng chậm.
- Khách hàng mới (104, 105) mang lại doanh thu cao.